# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
import PyPDF2
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://www.necc.org/wp-content/uploads/2015/08/AutismFAQ.pdf'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
print(file)
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df.head()

Excel/Source7_2.xlsx


,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/,https://autismcanada.org/autism-explained/cooccurring-conditions/,https://autismcanada.org/autism-explained/screening-tools/,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd),https://www.healthychildren.org/English/health-issues/conditions/Autism/Pages/default.aspx,https://www.ninds.nih.gov/Disorders/Patient-Caregiver-Education/Fact-Sheets/Autism-Spectrum-Disorder-Fact-Sheet#3082_5,https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/symptoms-causes/syc-20352928,https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/diagnosis-treatment/drc-20352934,https://www.necc.org/wp-content/uploads/2015/08/AutismFAQ.pdf
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Autism spectrum disorders (ASD) are a group of...,NaN,NaN,OverviewAutism spectrum disorder is a conditio...,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,https://autismcanada.org/autism-explained/diag...,NaN,NaN,NaN,It would be so much easier if \n autism spec...,\nASD symptoms can vary greatly from person to...,NaN,DiagnosisYour child's doctor will look for sig...,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,​Many children with \n autism spectrum disor...,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,"Autism Spectrum Disorder, or autism, is a neur...",NaN,NaN,NaN,NaN,\nAutism spectrum disorder (ASD) refers to a g...,NaN,NaN,NaN


# Scrape QA pairs from website

In [4]:
# creating a pdf file object
pdfFileObj = open('C:/Users/aparg/Downloads/AutismFAQ.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# extracting text from page
text = ''
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)    
    text = text + pageObj.extractText() + " "

# closing the pdf file object
pdfFileObj.close()

In [5]:
x = " ".join(text.split())

In [6]:
l1 = ['What causes autism?',
     'How many people have autism?',
     'What are some of the behaviors exhibited by a child who may have an autism spectrum disorder?',
     'My child is 3 and not talking yet, does this mean he has autism?',
     'What kinds of tests need to be done to diagnose autism?',
     'What treatment has been proven effective for treating autism?',
     'What is Applied Behavior Analysis?',
     'Why ABA?',
     'Who is qualified to provide ABA services?',
     'Does ABA always work?',
     'Does the MMR vaccine cause autism?',
     'Does mercury used in vaccines cause autism?',
     'Why do concerns about mercury in vaccines persist?',
     'Will there be any resolution?']

In [7]:
l2=[]
for i in range(len(l1)):
    start = x.index(l1[i])
    if(i!=len(l1)-1):
        end = x.index(l1[i+1])
    else:
        end = x.index('Americ')
    ans = x[start:end].strip()
    
    l2.append(ans)

In [8]:
len(l1),len(l2)

(14, 14)

In [9]:
for i in range(len(l2)):
    print(i,l1[i])
    print('-'*50)
    print(l2[i])
    print('*'*100)

0 What causes autism?
--------------------------------------------------
What causes autism? Genetic inheritance is involved in the development of autism spectrum disorders (ASD). Genetic factors likely interact with environmental variables to result in the expression of autism. Neurobiological research indicates that autism is likely the result of genetically determined abnormalities in brain development. This abnormal brain development may start before the child is born. It has been difficult for genetics researchers to locate a specific chromosome(s) involved in autism but recent studies have identified several genetic mechanisms that each lead to autism.
****************************************************************************************************
1 How many people have autism?
--------------------------------------------------
How many people have autism? The original estimates of the prevalence of autism indicated that it occurred in around 2 per 10 ,000 people. However, re

# Check which questions are similar

In [10]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [11]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [12]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [13]:
sim(l1)

Website -->  What causes autism? (Index 0)
100 questions list -->  What is Autism? (Index 4)
0.5
****************************************************************************************************
Website -->  How many people have autism? (Index 1)
100 questions list -->  What is Autism? (Index 4)
0.3333333333333333
****************************************************************************************************
Website -->  What are some of the behaviors exhibited by a child who may have an autism spectrum disorder? (Index 2)
100 questions list -->  What can parents do to help their child with an autism spectrum disorder? (Index 34)
0.2857142857142857
****************************************************************************************************
Website -->  My child is 3 and not talking yet, does this mean he has autism? (Index 3)
100 questions list -->  What can parents do to help their child with an autism spectrum disorder? (Index 34)
0.25
********************************

In [14]:
l2[3]

'My child is 3 and not talking yet, does this mean he has autism? No. There are a variety of disorders that are related to limited speech development but any child that is not talking by the age of 3 should have a developmental assessment immediately. Most children are babbling before they reach their first birthday and use single words to com municate by 18 months. Any child that does not display these behaviors or who has and subsequently loses a communicative or social skill should be screened for an autism spectrum disorder as soon as possible.'

In [15]:
# Website, 100Q
# 4, 21
# 5, 46
# 6, 10
# 10, 67

In [16]:
for i,j in enumerate(df['Question']):
    if('diag' in j.lower()):
        print(i,j)

1 How is ASD diagnosed?
21 What medical tests should the doctor perform when making a diagnosis of Autism?
61 How do doctors diagnose children with Autism?
63 What is a multidisciplinary team and how do they help to diagnose an autistic child?
64 Are their tests that are not recommended for the diagnosis or management of autism?
118 My child was just diagnosed with Autism. What do I do first?
124 I spent time and money childproofing my home when my son was an infant? Do I have to change anything now that he has been diagnosed as autistic?


In [17]:
# Directly add answer to existing dataframe
l4 = [[4, 21],
[5, 46],
[6, 10],
[10, 67]]

for i in l4:
    df[link].loc[i[1]] = l2[i[0]]

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [indexing.py:1732]


In [18]:
l1

['What causes autism?',
 'How many people have autism?',
 'What are some of the behaviors exhibited by a child who may have an autism spectrum disorder?',
 'My child is 3 and not talking yet, does this mean he has autism?',
 'What kinds of tests need to be done to diagnose autism?',
 'What treatment has been proven effective for treating autism?',
 'What is Applied Behavior Analysis?',
 'Why ABA?',
 'Who is qualified to provide ABA services?',
 'Does ABA always work?',
 'Does the MMR vaccine cause autism?',
 'Does mercury used in vaccines cause autism?',
 'Why do concerns about mercury in vaccines persist?',
 'Will there be any resolution?']

In [19]:
# update questions list
f = [i[0] for i in l4]
f = set(range(len(l1)))-set(f)
l1=[l1[i] for i in f]
l1

['What causes autism?',
 'How many people have autism?',
 'What are some of the behaviors exhibited by a child who may have an autism spectrum disorder?',
 'My child is 3 and not talking yet, does this mean he has autism?',
 'Why ABA?',
 'Who is qualified to provide ABA services?',
 'Does ABA always work?',
 'Does mercury used in vaccines cause autism?',
 'Why do concerns about mercury in vaccines persist?',
 'Will there be any resolution?']

In [20]:
l1=l1[:-1]

In [21]:
# select answers to questions in list 'l'
l2 = [l2[i] for i in f]

In [22]:
l2=l2[:-1]

In [23]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.necc.org/wp-content/uploads/2015/08/AutismFAQ.pdf
0,What causes autism?,What causes autism? Genetic inheritance is inv...
1,How many people have autism?,How many people have autism? The original esti...
2,What are some of the behaviors exhibited by a ...,What are some of the behaviors exhibited by a ...
3,"My child is 3 and not talking yet, does this m...","My child is 3 and not talking yet, does this m..."
4,Why ABA?,"Why ABA? No single ""treatment package"" will se..."
5,Who is qualified to provide ABA services?,Who is qualified to provide ABA services? The ...
6,Does ABA always work?,Does ABA always work? This is a difficult ques...
7,Does mercury used in vaccines cause autism?,Does mercury used in vaccines cause autism? Th...
8,Why do concerns about mercury in vaccines pers...,Why do concerns about mercury in vaccines pers...


In [24]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3.head(10)

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/,https://autismcanada.org/autism-explained/cooccurring-conditions/,https://autismcanada.org/autism-explained/screening-tools/,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd),https://www.healthychildren.org/English/health-issues/conditions/Autism/Pages/default.aspx,https://www.ninds.nih.gov/Disorders/Patient-Caregiver-Education/Fact-Sheets/Autism-Spectrum-Disorder-Fact-Sheet#3082_5,https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/symptoms-causes/syc-20352928,https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/diagnosis-treatment/drc-20352934,https://www.necc.org/wp-content/uploads/2015/08/AutismFAQ.pdf
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Autism spectrum disorders (ASD) are a group of...,NaN,NaN,OverviewAutism spectrum disorder is a conditio...,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,https://autismcanada.org/autism-explained/diag...,NaN,NaN,NaN,It would be so much easier if \n autism spec...,\nASD symptoms can vary greatly from person to...,NaN,DiagnosisYour child's doctor will look for sig...,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,​Many children with \n autism spectrum disor...,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,"Autism Spectrum Disorder, or autism, is a neur...",NaN,NaN,NaN,NaN,\nAutism spectrum disorder (ASD) refers to a g...,NaN,NaN,NaN
5,What are Screening Tools for Autism?,NaN,Early intervention can make a big difference f...,NaN,NaN,NaN,NaN,NaN,\nDevelopmental Screening ToolsScreening tools...,NaN,NaN,NaN,NaN,NaN,NaN
6,How Common Is It?,NaN,"For many years a diagnosis of autism was rare,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,What are the Signs of Autism?,NaN,Many signs can indicate that an individual may...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"\nEven as infants, children with ASD may seem ...",NaN,NaN,NaN
8,What should I do next?,NaN,When preparing to talk to your medical provide...,"Attend one or more parent support groups, o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,How can you tell Autism from Asperger’s Syndrome?,NaN,NaN,Asperger syndrome was phased out with the int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df3.shape

(146, 15)

In [26]:
df3.iloc[:,[0,-1]].dropna()

,Question,https://www.necc.org/wp-content/uploads/2015/08/AutismFAQ.pdf
10,What is Applied Behavioral Analysis?,What is Applied Behavior Analysis? Applied Beh...
21,What medical tests should the doctor perform w...,What kinds of tests need to be done to diagnos...
46,\nHow is autism treated,What treatment has been proven effective for t...
53,Does ABA always work?,Does ABA always work? This is a difficult ques...
54,Does mercury used in vaccines cause autism?,Does mercury used in vaccines cause autism? Th...
55,Is there a link between MMR Vaccine and the de...,Does the MMR vaccine cause autism? Scientific ...
56,How many people have autism?,How many people have autism? The original esti...
57,"My child is 3 and not talking yet, does this m...","My child is 3 and not talking yet, does this m..."
58,What are some of the behaviors exhibited by a ...,What are some of the behaviors exhibited by a ...
59,What causes autism?,What causes autism? Genetic inheritance is inv...


# Save Output

In [27]:
df3.to_excel('Excel/Source8.xlsx',index=False)